In [2]:
sc

In [3]:
aadt = 'aadt_gps_cleaned.csv'

In [4]:
import csv 
with open (aadt,'rb') as fi:
    reader = csv.reader(fi)
    print reader.next()
    print reader.next()

['', 'Roadway Begin Description', 'Roadway End Description', 'Road Name', 'AADT', 'latlng']
['0', 'NAVY ST', 'BROADWAY', 'PARK AVE', '10402', '40.6969737,-73.97945519999999']


In [5]:
df = spark.read \
    .format('com.databricks.spark.csv') \
    .option('header','true') \
    .option('inferSchema','true') \
    .option('parserLib','UNIVOCITY') \
    .load(aadt)

In [6]:
df.take(10)

[Row(_c0=0, Roadway Begin Description=u'NAVY ST', Roadway End Description=u'BROADWAY', Road Name=u'PARK AVE', AADT=10402, latlng=u'40.6969737,-73.97945519999999'),
 Row(_c0=1, Roadway Begin Description=u'CANAL ST', Roadway End Description=u'E 8TH ST', Road Name=u'MERCER ST', AADT=3304, latlng=u'40.7257515,-73.9976728'),
 Row(_c0=2, Roadway Begin Description=u'EBBITTS AVE', Roadway End Description=u'GARIBALDI AVE', Road Name=u'ROMA AVE', AADT=496, latlng=u'40.5611642,-74.1040998'),
 Row(_c0=3, Roadway Begin Description=u'4TH AVE', Roadway End Description=u'SECOND AVE', Road Name=u'65TH ST', AADT=3703, latlng=u'47.6759491,-122.32457'),
 Row(_c0=4, Roadway Begin Description=u'E FORDHAM RD', Roadway End Description=u'RT 907H BRONX RIVER PKWY SOU', Road Name=u'E FORDHAM RD', AADT=35426, latlng=u'40.8589734,-73.8862766'),
 Row(_c0=5, Roadway Begin Description=u'PELHAM PKWY BOSTON POST RD R', Roadway End Description=u'GUN HILL RD', Road Name=u'BOSTON RD', AADT=15079, latlng=u'40.8915608999999

In [7]:
aadt2 = sc.textFile(aadt,use_unicode=False).cache()

In [8]:
list(enumerate(aadt2.first().split(',')))

[(0, ''),
 (1, 'Roadway Begin Description'),
 (2, 'Roadway End Description'),
 (3, 'Road Name'),
 (4, 'AADT'),
 (5, 'latlng')]

In [9]:
noHeaderSat = sat.filter(lambda x: not x.startswith('DBN,'))
noHeaderSat.first()

NameError: name 'sat' is not defined

In [27]:
def extractCount(partitionId,partition):
    if partitionId == 0:
        partition.next()
    import csv
    reader = csv.reader(partition)
    for row in reader:
        if row[2]!='s':
            (latlng,vCount,keyCount,road)=(row[5],int(row[4]),1,str(row[1])+ ' ' +str(row[3]))
            yield (latlng,(vCount,keyCount,road))
            

carCount = aadt2.mapPartitionsWithIndex(extractCount)
carCount.take(10)

[('40.6969737,-73.97945519999999', (10402, 1, 'NAVY ST PARK AVE')),
 ('40.7257515,-73.9976728', (3304, 1, 'CANAL ST MERCER ST')),
 ('40.5611642,-74.1040998', (496, 1, 'EBBITTS AVE ROMA AVE')),
 ('47.6759491,-122.32457', (3703, 1, '4TH AVE 65TH ST')),
 ('40.8589734,-73.8862766', (35426, 1, 'E FORDHAM RD E FORDHAM RD')),
 ('40.89156089999999,-73.817953',
  (15079, 1, 'PELHAM PKWY BOSTON POST RD R BOSTON RD')),
 ('40.89649259999999,-73.8050574',
  (15126, 1, 'PELHAM PKWY BOSTON POST RD R BOSTON RD')),
 ('40.873512,-73.854012', (21199, 1, 'GUN HILL RD BOSTON RD')),
 ('40.873512,-73.854012', (21265, 1, 'GUN HILL RD BOSTON RD')),
 ('40.8657445,-73.9270717', (31007, 1, 'DYCKMAN ST BROADWAY'))]

In [29]:
# carCount.groupByKey().mapValues(lambda x: sum(x)/2).take(100)\
aadt_counts = carCount.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
    .mapValues(lambda x: x[0]/x[1]) \
    .collect()
aadt_counts

[('40.59831,-73.9857972', 3812),
 ('40.597848,-74.04616419999999', 11963),
 ('40.675265,-73.865689', 28963),
 ('40.623325,-74.113503', 8760),
 ('40.7604735,-73.9617509', 13106),
 ('40.5936411,-73.9973127', 9935),
 ('40.57884,-73.98435049999999', 23068),
 ('40.83609999999999,-73.92156', 13443),
 ('40.6606953,-73.7678257', 13423),
 ('40.7578587,-73.9132999', 1388),
 ('40.2097826,-76.9495018', 13405),
 ('40.5781731,-73.9534606', 8879),
 ('40.7436872,-73.71792889999999', 11278),
 ('40.8224024,-73.8526565', 3301),
 ('40.8825468,-73.8937357', 22163),
 ('40.7193867,-74.00661649999999', 15397),
 ('40.7401849,-73.925119', 4039),
 ('40.7390589,-73.9427178', 41524),
 ('40.7330314,-73.85420470000001', 4862),
 ('40.76043900000001,-73.994621', 8519),
 ('40.621994,-73.99492', 5470),
 ('40.7086312,-73.9507351', 8307),
 ('40.6176306,-74.10455080000001', 27832),
 ('40.7160251,-73.99755139999999', 2071),
 ('40.7554239,-73.97056789999999', 2830),
 ('41.5465685,-87.4616065', 14708),
 ('40.8735491,-73.87870

In [54]:
print aadt_counts[0][0]
print aadt_counts[0][1]

with open('aadt_counts.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for i in range (0,len(aadt_counts)):
        wr.writerow(aadt_counts[i])

        
    

40.59831,-73.9857972
3812
